In [1]:

# Barebones code to plot interactive correlation plot on web

In [39]:
def log(msg):
    with open("voila_debug_log.txt", "a", encoding="utf-8") as f:
        f.write(str(msg) + "\n")

# Optional: clear log at start of each session
with open("voila_debug_log.txt", "w") as f:
    f.write("Starting new Voila session...\n")


In [40]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import pearsonr



In [41]:
# Load the saved CSV file back into a dataframe
merged_data = pd.read_csv("merged_data_MAR_LOI_GS.csv", index_col=0)

In [42]:
# Automatically extract site names and variable types from column names
site_names = sorted(set(col.split('_')[0] for col in merged_data.columns if "_Year" in col))
variables = sorted(set(col.split('_')[1] for col in merged_data.columns if not col.endswith("Year")))


In [43]:
def update_plot(site1, site2, year_range, var1, var2):
    try:
        # Site 1
        df1 = merged_data[[f'{site1}_Year', f'{site1}_{var1}']].dropna().rename(columns={f'{site1}_Year': 'Year', f'{site1}_{var1}': 'X'})
        df1_full = df1.copy()
        df1_zoom = df1[(df1['Year'] >= year_range[0]) & (df1['Year'] <= year_range[1])]

        # Site 2
        df2 = merged_data[[f'{site2}_Year', f'{site2}_{var2}']].dropna().rename(columns={f'{site2}_Year': 'Year', f'{site2}_{var2}': 'Y'})
        df2_full = df2.copy()
        df2_zoom = df2[(df2['Year'] >= year_range[0]) & (df2['Year'] <= year_range[1])]

        # Pearson correlation only where both zoomed datasets align
        df_zoom_merged = pd.merge(df1_zoom, df2_zoom, on='Year')
        if len(df_zoom_merged) >= 2 and df_zoom_merged['X'].nunique() > 1 and df_zoom_merged['Y'].nunique() > 1:
            r, p = pearsonr(df_zoom_merged['X'], df_zoom_merged['Y'])
            corr_label = f"Pearson r = {r:.2f}, p = {p:.2g}"
        else:
            corr_label = "Correlation not valid"

        # Colors
        color1 = CUD_COLORS.get(var1, CUD_COLORS["default"])
        color2 = CUD_COLORS.get(var2, CUD_COLORS["default"])

        # Update traces
        fig.data[0].x = df1_full['Year']
        fig.data[0].y = df1_full['X']
        fig.data[0].name = f'{site1} {var1} (all)'
        fig.data[0].line.color = color1
        fig.data[0].line.dash = 'dot'

        fig.data[1].x = df2_full['Year']
        fig.data[1].y = df2_full['Y']
        fig.data[1].name = f'{site2} {var2} (all)'
        fig.data[1].line.color = color2
        fig.data[1].line.dash = 'dot'

        fig.data[2].x = df1_zoom['Year']
        fig.data[2].y = df1_zoom['X']
        fig.data[2].name = f'{site1} {var1} (zoom)'
        fig.data[2].line.color = color1
        fig.data[2].line.width = 4

        fig.data[3].x = df2_zoom['Year']
        fig.data[3].y = df2_zoom['Y']
        fig.data[3].name = f'{site2} {var2} (zoom)'
        fig.data[3].line.color = color2
        fig.data[3].line.width = 4

        # Update titles
        fig.update_layout(
            title=dict(
                text=f"{site1} {var1} vs {site2} {var2} ({year_range[0]}–{year_range[1]})<br>{corr_label}",
                x=0.5,
                xanchor='center'
            ),
            yaxis=dict(title=dict(text=f'{site1} {var1}', font=dict(color=color1)), tickfont=dict(color=color1)),
            yaxis2=dict(title=dict(text=f'{site2} {var2}', font=dict(color=color2)), tickfont=dict(color=color2)),
            annotations=[
                dict(
                    text="Correlations only valid when comparing variables within a site",
                    xref="paper", yref="paper",
                    x=0.5, y=1.1,  # above the main title
                    showarrow=False,
                    font=dict(size=10, color="black"),
                    align="center"
                )
            ]
        )


        display(HTML(plot(fig, include_plotlyjs=True, output_type='div')))

    except Exception as e:
        log(f"[ERROR] Exception in update_plot: {e}")



In [45]:
# Make the site dropdowns
site1_selector = widgets.Dropdown(
    options=site_names,
    value=site_names[0],
    description='Site 1:'
)

site2_selector = widgets.Dropdown(
    options=site_names,
    value=site_names[0],
    description='Site 2:'
)

# Variable dropdowns start EMPTY
var1_selector = widgets.Dropdown(
    options=[],
    description='Variable 1:'
)

var2_selector = widgets.Dropdown(
    options=[],
    description='Variable 2:'
)

# Update variable dropdowns when sites are changed
def update_var1(*args):
    site1 = site1_selector.value
    valid_vars = [v.replace(f"{site1}_", "") for v in merged_data.columns if v.startswith(f"{site1}_") and not v.endswith("_Year")]
    var1_selector.options = valid_vars
    if valid_vars:
        var1_selector.value = valid_vars[0]  # Set default

def update_var2(*args):
    site2 = site2_selector.value
    valid_vars = [v.replace(f"{site2}_", "") for v in merged_data.columns if v.startswith(f"{site2}_") and not v.endswith("_Year")]
    var2_selector.options = valid_vars
    if valid_vars:
        var2_selector.value = valid_vars[0]  # Set default

# Trigger variable updates on site change
site1_selector.observe(update_var1, 'value')
site2_selector.observe(update_var2, 'value')

# Initialize once
update_var1()
update_var2()

# Layout nicely
ui = widgets.VBox([
    widgets.HBox([site1_selector, var1_selector]),
    widgets.HBox([site2_selector, var2_selector]),
    year_slider
])

# Hook up to update_plot
out = widgets.interactive_output(
    update_plot,
    {
        'site1': site1_selector,
        'site2': site2_selector,
        'year_range': year_slider,
        'var1': var1_selector,
        'var2': var2_selector
    }
)

display(ui, out)


Output()